<a href="https://colab.research.google.com/github/Penguinbeanie/Capstone-Project/blob/main/Capstone_Project_Data_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import google.generativeai as genai
from IPython.display import display, Markdown
from google.colab import userdata
import json

# retrieving the key stored in Colab
key = userdata.get('GOOGLE_API_KEY')

# configure the key for calling GenAI model
genai.configure(api_key=key)

# load model
model = genai.GenerativeModel("gemini-1.5-flash")

In [4]:
import json
import requests
from google.colab import files

# List of input files (replace with your GitHub raw URLs)
input_files = [
    "https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_files/questionnaire1.json",
    "https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_files/questionnaire2.json",
    "https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_files/questionnaire3.json",
    "https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_files/questionnaire4.json",
    "https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_files/questionnaire5.json"
]

# Dictionary to hold questions grouped by type
question_types = {}

# Iterate through each file
for file_url in input_files:
    response = requests.get(file_url)
    data = response.json()
    for entry in data:
        q_type = entry['type']
        if q_type not in question_types:
            question_types[q_type] = []
        question_types[q_type].append(entry)

# Save each question type to separate JSON files
output_files = []
for q_type, questions in question_types.items():
    filename = f"/content/{q_type}.json"
    with open(filename, 'w') as f:
        json.dump(questions, f, indent=4)
    output_files.append(filename)

print("Files have been created.")


Files have been created.


In [6]:
# Load files

import json
import requests

# GitHub raw URLs (you'll need to replace these with your actual GitHub raw URLs)
file_path_date = 'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_seperation/DATE.json'
file_path_multi = 'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_seperation/MULTI_SELECT.json'
file_path_number = 'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_seperation/NUMBER.json'
file_path_single = 'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_seperation/SINGLE_SELECT.json'
file_path_text = 'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_seperation/TEXT.json'

# Function to load JSON from URL
def load_json_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes
    return response.json()

# Load JSON data from URLs
try:
    json_data_date = load_json_from_url(file_path_date)
    json_data_multi = load_json_from_url(file_path_multi)
    json_data_number = load_json_from_url(file_path_number)
    json_data_single = load_json_from_url(file_path_single)
    json_data_text = load_json_from_url(file_path_text)

    # Convert JSON data to strings (if needed)
    json_string_date = json.dumps(json_data_date, indent=4)
    json_string_multi = json.dumps(json_data_multi, indent=4)
    json_string_number = json.dumps(json_data_number, indent=4)
    json_string_single = json.dumps(json_data_single, indent=4)
    json_string_text = json.dumps(json_data_text, indent=4)

except requests.exceptions.RequestException as e:
    print(f"Error loading JSON files: {e}")


Error loading JSON files: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_seperation/DATE.json


In [7]:
# prompt

multiVar = 10
singleVar = 10
numberVar = 10
textVar = 10
dateVar = 10

prompt_multi = f"""
                Using the context below as a template, create {multiVar} more JSON objects. The output should include only the keys "type",
                "question", and "options". Ensure the "type" is always "MULTI-SELECT". The "options" array should contain multiple unique objects,
                each with the "option" key and a meaningful value representing an available choice. No IDs should be included.
                Generate multiple unique examples of questions that make sense in context for the type "MULTI-SELECT".
                Output only valid JSON, with proper line spacing and indentation, without any additional formatting or code block delimiters.

                Topic: Diverse

                Context:
                {json_string_single}
                """

prompt_single = f"""
                Using the context below as a template, create {singleVar} more JSON objects. The output should include only the keys "type",
                "question", and "options". Ensure the "type" is always "SINGLE-SELECT". The "options" array should contain multiple unique objects,
                each with the "option" key and a meaningful value representing an available choice. No IDs should be included.
                Generate multiple unique examples of questions that make sense in context for the type "SINGLE-SELECT".
                Output only valid JSON, with proper line spacing and indentation, without any additional formatting or code block delimiters.

                Topic: Communications

                Context:
                {json_string_single}
                """

prompt_text = f"""
                Using the context below as a template, create {textVar} more JSON objects. The output should include only the keys "type",
                "question", and "options". Ensure the "type" is always "TEXT", the "options" array always contains exactly one object with
                the "option" set to "Text", and no IDs are included. Generate multiple unique examples of questions that make sense in context
                for the type "TEXT". Output only valid JSON, with proper line spacing and indentation, without any additional formatting or code block delimiters.

                Context:
                {json_string_text}
                """

prompt_number = f"""
                Using the context below as a template, create {numberVar} more JSON objects. The output should include only the keys "type",
                "question", and "options". Ensure the "type" is always "NUMBER", the "options" array always contains exactly one object with
                the "option" set to "Number", and no IDs are included. Generate multiple unique examples of questions that make sense in context
                for the type "NUMBER". Output only valid JSON, with proper line spacing and indentation, without any additional formatting or code block delimiters.

                Context:
                {json_string_number}
                """

prompt_date = f"""
                Using the context below as a template, create {dateVar} more JSON objects. The output should include only the keys "type",
                "question", and "options". Ensure the "type" is always "DATE", the "options" array always contains exactly one object with
                the "option" set to "Date", and no IDs are included. Generate multiple unique examples of questions that make sense in context
                for the type "DATE". Output only valid JSON, with proper line spacing and indentation, without any additional formatting or code block delimiters.

                Context:
                {json_string_date}
                """


# responses
responses = {
    #"prompt_multi": "Questionnaire_Multi_Artificial.json",
    "prompt_single": "Questionnaire_Single_Artificial.json"#,
    #"prompt_date": "Questionnaire_Date_Artificial.json",
    #"prompt_number": "Questionnaire_Number_Artificial.json",
    #"prompt_text": "Questionnaire_Text_Artificial.json"
}

for response_name, response_file in responses.items():
    prompt = globals()[response_name]  # Get the prompt string
    response = model.generate_content(prompt)  # Get model's response

    print(response.text)

    try:
        response_data = json.loads(response.text)  # Parse the response text as JSON
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for {response_name}: {e}")
        continue

    # Write the parsed JSON to a file
    with open(response_file, "w") as json_file:
        json.dump(response_data, json_file, indent=4)

    #print(f"JSON file '{response_file}' has been created successfully!")

[
    {
        "type": "SINGLE-SELECT",
        "question": "Preferred method of contact",
        "options": [
            {
                "option": "Email"
            },
            {
                "option": "Phone"
            },
            {
                "option": "Mail"
            }
        ]
    },
    {
        "type": "SINGLE-SELECT",
        "question": "How did you hear about us?",
        "options": [
            {
                "option": "Online Advertisement"
            },
            {
                "option": "Referral"
            },
            {
                "option": "Trade Show"
            },
            {
                "option": "Social Media"
            }
        ]
    },
    {
        "type": "SINGLE-SELECT",
        "question": "What is your communication style?",
        "options": [
            {
                "option": "Direct and concise"
            },
            {
                "option": "Detailed and thorough"
            },
   

#JSON to Dataframe

###MULTI

In [8]:
import json
import pandas as pd
import requests

#Load the JSON data
questions = [
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/MULTI/Questionnaire_Multi_Artificial_Diverse.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/MULTI/Questionnaire_Multi_Artificial_HeavyIndustry.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/MULTI/Questionnaire_Multi_Artificial_Sales.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/MULTI/Questionnaire_Multi_Artificial_SoftwareDev.json'
]

dfs = []

for question in questions:
    response = requests.get(question)
    data = response.json()

    #Create a DataFrame from the JSON data
    df_temp = pd.DataFrame(data)
    df_temp['options'] = df_temp['options'].apply(lambda x: [item['option'] for item in x])
    dfs.append(df_temp)

df = pd.concat(dfs, ignore_index=True)

df_for_csv = df.copy()
df_for_csv['options'] = df_for_csv['options'].apply(lambda x: '; '.join(x))
df_for_csv.to_csv('MULTI_combined.csv', index=False)

df

,type,question,options
0,MULTI_SELECT,Which social media platforms do you use for ma...,"[Facebook, Instagram, Twitter, LinkedIn, TikTo..."
1,MULTI-SELECT,What types of marketing content do you find mo...,"[Blog posts, Videos, Infographics, Case studie..."
2,MULTI-SELECT,What are your primary goals for this marketing...,"[Increase brand awareness, Generate leads, Dri..."
3,MULTI-SELECT,What are your preferred methods of customer co...,"[Email, Phone, Chat, Social media, In-person m..."
4,MULTI-SELECT,What challenges are you facing in your current...,"[Lack of budget, Lack of time, Lack of resourc..."
...,...,...,...
186,MULTI-SELECT,What types of software development tools have ...,"[Integrated Development Environments (IDEs), S..."
187,MULTI-SELECT,What types of communication protocols are you ...,"[HTTP, HTTPS, TCP/IP, UDP, REST, GraphQL]"
188,MULTI-SELECT,What types of design patterns have you impleme...,"[Creational patterns, Structural patterns, Beh..."
189,MULTI-SELECT,What are your preferred methods for maintainin...,"[Code reviews, Static analysis, Unit testing, ..."


###SINGLE

In [9]:
import json
import pandas as pd

#Load the JSON data

#Load the JSON data

questions = [
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/SINGLE/Questionnaire_Single_Artificial_ArtIndustry.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/SINGLE/Questionnaire_Single_Artificial_Communications.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/SINGLE/Questionnaire_Single_Artificial_Diverse.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/SINGLE/Questionnaire_Single_Artificial_SoftwareDev.json'
]

dfs = []

for question in questions:
    response = requests.get(question)
    data = response.json()

    #Create a DataFrame from the JSON data

    df_temp = pd.DataFrame(data)
    df_temp['options'] = df_temp['options'].apply(lambda x: [item['option'] for item in x])
    dfs.append(df_temp)

df = pd.concat(dfs, ignore_index=True)

df_for_csv = df.copy()
df_for_csv['options'] = df_for_csv['options'].apply(lambda x: '; '.join(x))
df_for_csv.to_csv('SINGLE_combined.csv', index=False)

df


,type,question,options
0,SINGLE-SELECT,What is your preferred medium for artistic exp...,"[Painting, Sculpture, Digital Art, Photography..."
1,SINGLE-SELECT,What style of art are you most drawn to?,"[Abstract, Realism, Surrealism, Impressionism,..."
2,SINGLE-SELECT,What is your primary role in the art and desig...,"[Artist, Designer, Art Director, Graphic Desig..."
3,SINGLE-SELECT,What software do you primarily use for your work?,"[Adobe Photoshop, Adobe Illustrator, Adobe InD..."
4,SINGLE-SELECT,What is your preferred color palette?,"[Warm Colors, Cool Colors, Monochromatic, Anal..."
...,...,...,...
145,SINGLE-SELECT,What is your experience with data streaming te...,"[Beginner, Intermediate, Advanced]"
146,SINGLE-SELECT,What is your experience with big data technolo...,"[Beginner, Intermediate, Advanced]"
147,SINGLE-SELECT,What is your experience with artificial intell...,"[Beginner, Intermediate, Advanced]"
148,SINGLE-SELECT,What is your experience with blockchain techno...,"[Beginner, Intermediate, Advanced]"


# **Helper code**

In [ ]:
'''
import json
from google.colab import files

# List of input files
input_files = [
    "/content/questionnaire1.json",
    "/content/questionnaire2.json",
    "/content/questionnaire3.json",
    "/content/questionnaire4.json",
    "/content/questionnaire5.json"
]

# Dictionary to hold questions grouped by type
question_types = {}

# Iterate through each file
for file in input_files:
    with open(file, 'r') as f:
        data = json.load(f)
        for entry in data:
            q_type = entry['type']
            if q_type not in question_types:
                question_types[q_type] = []
            question_types[q_type].append(entry)

# Save each question type to separate JSON files
output_files = []
for q_type, questions in question_types.items():
    filename = f"/content/{q_type}.json"
    with open(filename, 'w') as f:
        json.dump(questions, f, indent=4)
    output_files.append(filename)

# Provide download links for each file
for file in output_files:
    files.download(file)

print("Filtered question files have been created and are ready for download.")


SyntaxError: incomplete input (<ipython-input-4-767896fbdd23>, line 1)

In [ ]:
'''
import pandas as pd
from transformers import pipeline

# Initialize the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Example DataFrame
data = {
    "text": [
        "I prefer to eat mostly bread.",
        "I love grilled salmon.",
        "Steak is my favorite dish.",
        "I enjoy cheeseburgers."
    ],
    "labels": [["Bread", "Fish", "Meat", "Burger"],  # Labels for each row
               ["Fish", "Meat", "Bread", "Burger"],
               ["Meat", "Burger", "Fish", "Bread"],
               ["Burger", "Meat", "Fish", "Bread"]]
}

df_test = pd.DataFrame(data)

df_test


# Function to classify each row
def classify_row(row):
    result = classifier(row["text"], candidate_labels=row["labels"])
    return result["labels"][0], result["scores"][0]


# Apply the classification to each row

df["predicted_labels"], df["scores"] = zip(*df.apply(lambda row: classify_row(row), axis=1))

# Display the DataFrame
print(df)
'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


'\n# Function to classify each row\ndef classify_row(row):\n    result = classifier(row["text"], candidate_labels=row["labels"])\n    return result["labels"][0], result["scores"][0]\n\n\n# Apply the classification to each row\n\ndf["predicted_labels"], df["scores"] = zip(*df.apply(lambda row: classify_row(row), axis=1))\n\n# Display the DataFrame\nprint(df)\n'

In [ ]:
'''
import pandas as pd

data_test = {
    "text": [
        "I prefer to eat mostly bread.",
        "I love grilled salmon.",
        "Steak is my favorite dish.",
        "I enjoy cheeseburgers."
    ],
    "labels": [["Bread", "Fish", "Meat", "Burger"],  # Labels for each row
               ["Fish", "Meat", "Bread", "Burger"],
               ["Meat", "Burger", "Fish", "Bread"],
               ["Burger", "Meat", "Fish", "Bread"]]
}

df_test = pd.DataFrame(data)

df_test

print(df)
'''

'\nimport pandas as pd \n\ndata_test = {\n    "text": [\n        "I prefer to eat mostly bread.",\n        "I love grilled salmon.",\n        "Steak is my favorite dish.",\n        "I enjoy cheeseburgers."\n    ],\n    "labels": [["Bread", "Fish", "Meat", "Burger"],  # Labels for each row\n               ["Fish", "Meat", "Bread", "Burger"],\n               ["Meat", "Burger", "Fish", "Bread"],\n               ["Burger", "Meat", "Fish", "Bread"]]\n}\n\ndf_test = pd.DataFrame(data)\n\ndf_test\n\nprint(df)\n'